# 목차

* 평가기준
* 8가지 분류 모델 비교
* 딥러닝 모델
* 회고
* Reference

# 평가기준

루브릭

아래의 기준을 바탕으로 프로젝트를 평가합니다.

---
평가문항	상세기준
1. 분류 모델의 accuracy가 기준 이상 높게 나왔는가?

> 3가지 단어 개수에 대해 8가지 머신러닝 기법을 적용하여 그중 최적의 솔루션을 도출하였다.

2. 분류 모델의 F1 score가 기준 이상 높게 나왔는가?

> Vocabulary size에 따른 각 머신러닝 모델의 성능변화 추이를 살피고, 해당 머신러닝 알고리즘의 특성에 근거해 원인을 분석하였다.

3. 딥러닝 모델을 활용해 성능이 비교 및 확인되었는가?

> 동일한 데이터셋과 전처리 조건으로 딥러닝 모델의 성능과 비교하여 결과에 따른 원인을 분석하였다.

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import sklearn as sk
from tensorflow.keras.datasets import reuters

In [2]:
from sklearn.naive_bayes import MultinomialNB #다항분포 나이브 베이즈 모델
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score #정확도 계산

In [3]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# 8가지 분류 모델 비교

사용할 모델

나이브 베이즈 분류기, CNB, 로지스틱 회귀, 서포트 벡터 머신, 결정 트리, 랜덤 포레스트, 그래디언트 부스팅 트리, 보팅

In [4]:
word_index = reuters.get_word_index(path="reuters_word_index.json")
index_to_word = { index+3 : word for word, index in word_index.items() }
# index_to_word에 숫자 0은 <pad>, 숫자 1은 <sos>, 숫자 2는 <unk>를 넣어줍니다.
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token

550378/550378 [==============================] - 0s 0us/step


## 데이터 로드, num = None

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

num = None # num 값을 여러 수치로 바꿔보자

(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=num, test_split=0.2)

decoded_train = []
for i in range(len(x_train)):
  t = ' '.join([index_to_word[index] for index in x_train[i]])
  decoded_train.append(t)

x_train = decoded_train

decoded_test = []
for i in range(len(x_test)):
  t = ' '.join([index_to_word[index] for index in x_test[i]])
  decoded_test.append(t)

x_test = decoded_test

### 분류 모델 생성 및 학습

In [ ]:
dtmvector = CountVectorizer()
tfidf_transformer = TfidfTransformer()

x_train_dtm = dtmvector.fit_transform(x_train) # 학습 데이터를 DTM으로 변환
tfidfv = tfidf_transformer.fit_transform(x_train_dtm) # DTM을 TF-IDF 행렬로 변환

x_test_dtm = dtmvector.transform(x_test) # 테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(x_test_dtm) # DTM을 TF-IDF 행렬로 변환

nbc = MultinomialNB()
nbc.fit(tfidfv, y_train)

nbc_predicted = nbc.predict(tfidfv_test) #테스트 데이터에 대한 예측

cb = ComplementNB()
cb.fit(tfidfv, y_train)

cb_predicted = cb.predict(tfidfv_test) #테스트 데이터에 대한 예측

lr = LogisticRegression(C=10000, penalty='l2', max_iter=3000)
lr.fit(tfidfv, y_train)

lr_predicted = lr.predict(tfidfv_test) #테스트 데이터에 대한 예측

lsvc = LinearSVC(C=1000, penalty='l1', max_iter=3000, dual=False)
lsvc.fit(tfidfv, y_train)

lsvc_predicted = lsvc.predict(tfidfv_test) #테스트 데이터에 대한 예측

tree = DecisionTreeClassifier(max_depth=10, random_state=0)
tree.fit(tfidfv, y_train)

tree_predicted = tree.predict(tfidfv_test) #테스트 데이터에 대한 예측

forest = RandomForestClassifier(n_estimators=5, random_state=0)
forest.fit(tfidfv, y_train)

forest_predicted = forest.predict(tfidfv_test) #테스트 데이터에 대한 예측

grbt = GradientBoostingClassifier(random_state=0) # verbose=3
grbt.fit(tfidfv, y_train)

grbt_predicted = grbt.predict(tfidfv_test) #테스트 데이터에 대한 예측
  
voting_classifier = VotingClassifier(estimators=[
         ('lr', LogisticRegression(C=10000, max_iter=3000, penalty='l2')),
        ('cb', ComplementNB()),
        ('grbt', GradientBoostingClassifier(random_state=0))], voting='soft')
voting_classifier.fit(tfidfv, y_train)
  
voting_classifier_predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
# 결과 확인 
print("정확도:", accuracy_score(y_test, nbc_predicted), "나이브 베이즈 분류기")
print("정확도:", accuracy_score(y_test, cb_predicted), "CNB")
print("정확도:", accuracy_score(y_test, lr_predicted), "로지스틱 회귀")
print("정확도:", accuracy_score(y_test, lsvc_predicted), "서포트 벡터 머신")
print("정확도:", accuracy_score(y_test, tree_predicted), "결정 트리")
print("정확도:", accuracy_score(y_test, forest_predicted), "랜덤 포레스트")
print("정확도:", accuracy_score(y_test, grbt_predicted), "그래디언트 부스팅 트리")
print("정확도:", accuracy_score(y_test, voting_classifier_predicted), "보팅")

# 딥러닝 모델

RNN 모델과 LSTM 레이어를 사용한다.

## num = None 일 때

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

num = None # num 값을 여러 수치로 바꿔보자

(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=num, test_split=0.2)

x_train = pad_sequences(x_train, truncating='post', maxlen=500)
x_test = pad_sequences(x_test, truncating='post', maxlen=500)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                    test_size=0.2, 
                                                    random_state=34)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
vocab_size = num    # 어휘 사전의 크기
word_vector_dim = 128  # 단어 하나를 표현하는 임베딩 벡터의 차원수 (변경가능)

from tensorflow import keras

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size,word_vector_dim))
model.add(keras.layers.LSTM(128))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(46, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_val, y_val))

In [ ]:
lstm_result = model.evaluate(x_test, y_test, verbose=2)
print(lstm_result)

# 회고

이번 노드에서는 단어장의 크기에 따른 모델의 성능 변화를 확인했다. 그리고 8가지의 분류 모델과 함께 RNN 딥러닝 모델과도 비교하는 시간을 가졌다. 실제로 비교하기 앞서 사전에 학습한 reference 내용에 따르면 단어장의 크기가 너무 커지면 얼마 나오지도 않는 단어 때문에 학습을 방해할 수 있다고 한다. 그리고 작은 크기의 데이터셋은 회귀 모델보다 분류 모델에 더욱 유리하다. 우리가 쓰는 로이터 뉴스 데이터셋의 크기는 작은 편에 속하므로 회귀 모델, 딥러닝 모델보다 분류 모델에 속해있는 랜덤 포레스트 등의 모델이 더욱 효과적일 것으로 예상할 수 있다. 또한 데이터셋의 크기가 작으면 단어장의 크기가 특정 부분일 때 최대 성능을 내고 그 전후에는 성능이 내려가는 특성이 커진다. 반대로 데이터셋의 크기가 매우 크면 단어장의 크기가 클수록 모델 성능이 증가한다.

따라서 예상할 수 있는 결과는 분류 모델이 가장 높은 성능을 내며 단어 장의 크기가 너무 작지도, 크지도 않은 사이즈일 때 최대 성능이 나오는 것이다. 단어장의 일정 크기를 넘어가면 점점 모델의 성능이 떨어지게 될 것이다.

이 로이터 뉴스 데이터셋은 46개의 클래스를 가지고 있는데 그 중에서 3,4번 등의 일부 클래스가 뉴스의 대부분을 차지하고 있는 불균형한 데이터셋이었다. 따라서 이런 경우 모델이 제대로 학습하기 어려울 수 있다. 이를 해결하기 위해서 손실함수를 바꿔보는 것을 시도할 수 있다. focal loss 라는 함수를 손실함수로 두면 불균형한 데이터셋 하에서의 모델 성능이 다른 손실함수에 비해 증가하기 한다. 때문에 만약 딥러닝 모델의 성능을 향상 시켜야만 하는 경우였다면 손실함수를 focal loss 로 바꾸었을 것이다. 하지만 오늘 노드는 어쨌거나 성능의 비교에 그 목적이 있으므로 손실함수를 focal loss 가 아닌 categorical crossentropy 로 두었다.

아쉽게도 한 번 학습을 진행시키는 데에 시간이 너무나도 오래 걸려 비교를 제대로 하지 못 했다. 다음에 기회가 된다면 마저 해보고 싶다.

# Reference

https://wikidocs.net/48558 RNN 에 대해

https://wikidocs.net/83544 padding 패딩

https://stellarway.tistory.com/45 적절한 vocab size